# UST Arbitrage

In [1]:
import binance_helpers as bh
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ui
import time
import importlib
from binance.enums import *

In [2]:
client = bh.new_binance_client()

In [3]:
t = client.get_all_tickers()

In [4]:
def get_price_of(prices, symbol):
    return float([d["price"] for d in prices if d['symbol']==symbol.upper()][0])

In [5]:
ethusdt = get_price_of(t, "ethusdt")

In [6]:
ethusdt

2013.65

In [41]:
a_usdt_dp = bh.get_decimal_place(client, "ethusdt")
b_usdt_dp = bh.get_decimal_place(client, "ustusdt")
ab_dp = bh.get_decimal_place(client, "ethust")

In [7]:
def trade(symbol, side, amt):
    print(f"{symbol}: {side}, amt: {amt}")
    if side==SIDE_BUY:
        order = client.create_order(
            symbol=symbol,
            side=SIDE_BUY,
            type=ORDER_TYPE_MARKET,
            quoteOrderQty=amt)
        return order
    elif side==SIDE_SELL:
        order = client.create_order(
            symbol=symbol,
            side=SIDE_SELL,
            type=ORDER_TYPE_MARKET,
            quantity=amt)
        return order
    else:
        print("ERROR")

In [100]:
def circular_order(a, b, usdt, a_usdt_dp, b_usdt_dp, ab_dp, a_dust=0, b_dust=0):
    """b is stablecoin"""
    print("Trading")
    # t = client.get_all_tickers()
    order = trade(f"{b}USDT", SIDE_BUY, usdt)
    order = client.get_order(symbol=f"{b}USDT",orderId=order['orderId'])
    
    b_amt = float(order['executedQty'])+b_dust
    order = trade(f"{a}{b}", SIDE_BUY, bh.binance_floor(b_amt, ab_dp))
    order = client.get_order(symbol=f"{a}{b}",orderId=order['orderId'])
    
    a_amt = float(order['executedQty'])+a_dust
    order = trade(f"{a}USDT", SIDE_SELL, bh.binance_floor(a_amt, a_usdt_dp))

In [8]:
def inverse_circular_order(a, b, usdt, a_usdt_dp, b_usdt_dp, ab_dp, a_dust=0, b_dust=0):
    """b is stablecoin"""
    print("Trading inverse")
    # t = client.get_all_tickers()
    # ap = get_price_of(t, f"{a}usdt")
    # bp = get_price_of(t, f"{b}usdt")
    order = trade(f"{a}USDT", SIDE_BUY, usdt)
    order = client.get_order(symbol=f"{a}USDT",orderId=order['orderId'])
    
    a_amt = float(order['executedQty'])+a_dust
    order = trade(f"{a}{b}", SIDE_SELL, bh.binance_floor(a_amt, ab_dp))
    order = client.get_order(symbol=f"{a}{b}",orderId=order['orderId'])
    
    b_amt = float(order['cummulativeQuoteQty'])+b_dust
    order = trade(f"{b}USDT", SIDE_SELL, bh.binance_floor(b_amt, b_usdt_dp))

In [53]:
t_now = time.time()
circular_order("ETH", "UST", 19, a_usdt_dp, b_usdt_dp, ab_dp)
print(time.time()-t_now)

ap: 1933.66, bp: 0.378156, abp: 5124.96
b_amt: 50.0, going to trade 0.0096
a_amt: 0.0096
0.8268189430236816


In [9]:
from IPython.display import clear_output

In [10]:
def get_amt(asset):
    return float(client.get_asset_balance(asset=asset)["free"])

In [16]:
a = "ETH"
b = "BTC"
a_usdt_dp = bh.get_decimal_place(client, f"{a}usdt")
b_usdt_dp = bh.get_decimal_place(client, f"{b}usdt")
ab_dp = bh.get_decimal_place(client, f"{a}{b}")
usdt = 15
just_traded = False
for _ in range(10000):
    t = client.get_all_tickers()
    if just_traded:
        a_dust = get_amt(a)
        b_dust = get_amt(b)
        just_traded = False
    ethusdt = get_price_of(t, f"{a}usdt")
    ustusdt = get_price_of(t, f"ustusdt")
    ethust = get_price_of(t, f"{a}ust")
    off = (ethust*ustusdt-ethusdt)*100/ethusdt
    print(f"market is off by: {round(off, 2)}%")
    if off < -2:
        # circular_order(a, b, usdt, a_usdt_dp, b_usdt_dp, ab_dp, a_dust, b_dust)
        just_traded = True
    elif off > 2:
        # inverse_circular_order(a, b, usdt, a_usdt_dp, b_usdt_dp, ab_dp, a_dust, b_dust)
        just_traded = True
    else:
        time.sleep(1)
        clear_output()

market is off by: 0.2%


KeyboardInterrupt: 